In [ ]:
from pathlib import Path
import pandas as pd
from pgscen.command_line import (
    load_solar_data, split_actuals_hist_future, split_forecasts_hist_future)


cur_path = Path("day-ahead_load_scenario.ipynb").parent.resolve()
data_dir = Path(cur_path, '..', "data").resolve()
(solar_site_actual_df, solar_site_forecast_df,
     solar_meta_df) = load_solar_data()

scenario_start_time = pd.to_datetime('2018-09-01 06:00:00',utc=True)
scen_timesteps = pd.date_range(start=scenario_start_time,
                               periods=24, freq='H')

(solar_site_actual_hists,
     solar_site_actual_futures) = split_actuals_hist_future(
            solar_site_actual_df, scen_timesteps)

(solar_site_forecast_hists,
     solar_site_forecast_futures) = split_forecasts_hist_future(
            solar_site_forecast_df, scen_timesteps)

In [ ]:
from pgscen.engine import SolarGeminiEngine

scenario_count = 1000

se = SolarGeminiEngine(solar_site_actual_hists,
                       solar_site_forecast_hists,
                       scenario_start_time, solar_meta_df)

se.fit_solar_model()
se.create_solar_scenario(scenario_count, solar_site_forecast_futures)

In [ ]:
out_dir = Path(data_dir, '..')
se.write_to_csv(out_dir, {'solar': solar_site_actual_futures},
                write_forecasts=True)

df = pd.read_csv('{}/20180901/solar/Adamstown_Solar.csv'.format(out_dir))
print(df)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [19, 11]

plt_ts = se.scen_timesteps[7]
print(plt_ts)
_ = plt.hist(se.scenarios['solar']['Adamstown Solar'][plt_ts], bins=30)

In [ ]:
plt_asset, plt_ts = tuple(se.gemini_dict['day']['gemini_model'].conditional_gpd_dict.keys())[311]
print('   '.join([plt_asset, str(plt_ts)]))

_ = plt.hist(
    se.gemini_dict['day']['gemini_model'].conditional_gpd_dict[
        plt_asset, plt_ts].slots['data'],
    bins=30
    )